# Messages
Messages are the fundamental unit of context for models in LangChain. They represent the input and output of models, carrying both the content and metadata needed to represent the state of a conversation when interacting with an LLM.

## Basic Usage

In [7]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage

model = init_chat_model("openai:gpt-5-nano")

In [8]:
from langchain.agents import create_agent

agent = create_agent(
    model=model, tools=[], system_prompt="You are a full-stack comedian"
)

In [9]:
human_msg = HumanMessage("Hello, how are you?")

result = agent.invoke({"messages": [human_msg]})

In [10]:
print(result["messages"][-1].content)

Hey there! I’m good—running on caffeine, code, and a fresh batch of jokes. How about you? Want a quick tech joke or chat about something you’re building? 
Here’s a warm-up: Why do programmers confuse Halloween and Christmas? Because Oct 31 == Dec 25.


In [11]:
print(type(result["messages"][-1]))

<class 'langchain_core.messages.ai.AIMessage'>


In [12]:
for msg in result["messages"]:
    print(f"{msg.type}: {msg.content}\n")

human: Hello, how are you?

ai: Hey there! I’m good—running on caffeine, code, and a fresh batch of jokes. How about you? Want a quick tech joke or chat about something you’re building? 
Here’s a warm-up: Why do programmers confuse Halloween and Christmas? Because Oct 31 == Dec 25.



> question to Sydney?: Why don't I see the system message here?

### Altenative formats
#### Strings

In [13]:
agent = create_agent(
    model=model,
    tools=[],
    system_prompt="You are a terse sports poet.",  # This is a SystemMessage under the hood
)

In [14]:
result = agent.invoke({"messages": "Tell me about baseball"})
print(result["messages"][-1].content)

Baseball: nine innings of quiet duel on a diamond, where a bat meets a ball and history watches.

- Field: a diamond with four bases, chalk lines, fair vs. foul, night-sky and sunlit grass.
- Teams: offense tries to score; defense tries to stop them with glove and arm.
- Roles: pitcher, catcher, infielders, outfielders; the batter stands, the crowd waits.
- Rhythm: at bat, pitch, field, throw, the slow pull of the moment toward an out.
- Rules in brief: three outs ends a half-inning; nine innings (often) per game; extras if tied.
- Scoring: run scores when a runner touches home plate after circling the bases.
- Plays you’ll hear: strike, ball, walk; hits—single, double, triple, home run; outs via grounder, flyout, strikeout.
- Strategy: power and patience, speed and placement, shifts and signs, the chess of the field.
- History in a breath: from dusty diamonds of the 19th century to Ruth’s thunder, Robinson’s courage, and today’s analytics.
- Why we love it: the crack of the bat, the d

#### Dictionaries

In [15]:
result = agent.invoke(
    {"messages": {"role": "user", "content": "Write a haiku about sprinters"}}
)
print(result["messages"][-1].content)

Lanes glare, feet ignite
Breaths drum, the track tightens fast
Line nears, breath erupts


There are multiple roles:
```python
messages = [
    {"role": "system", "content": "You are a sports poetry expert who completes haikus that have been started"},
    {"role": "user", "content": "Write a haiku about sprinters"},
    {"role": "assistant", "content": "Feet don't fail me..."}
]
```

## Output Format
### messages

In [16]:
from langchain_core.tools import tool


@tool
def check_haiku_lines(text: str):
    """Check if the given haiku text has exactly 3 lines.

    Returns None if it's correct, otherwise an error message.
    """
    # Split the text into lines, ignoring leading/trailing spaces
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    print(f"checking haiku, it has {len(lines)} lines:\n {text}")

    if len(lines) != 3:
        return f"Incorrect! This haiku has {len(lines)} lines. A haiku must have exactly 3 lines."
    return "Correct, This haiku has 3 lines."

In [17]:
agent = create_agent(
    model="openai:gpt-5",
    tools=[check_haiku_lines],
    system_prompt="You are a sports poet who only writes Haiku. You always check your work.",
)

In [18]:
result = agent.invoke({"messages": "Please write me a poem"})

checking haiku, it has 3 lines:
 Sneakers kiss the court
Orange arc meets hush and hope
Nets breathe, crowd erupts


In [19]:
result["messages"][-1].content

'Sneakers kiss the court\nOrange arc meets hush and hope\nNets breathe, crowd erupts'

In [20]:
print(len(result["messages"]))

4


In [21]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Please write me a poem
================================== Ai Message ==================================
Tool Calls:
  check_haiku_lines (call_O4GrvFp6i55zxaUxCZWEXxGv)
 Call ID: call_O4GrvFp6i55zxaUxCZWEXxGv
  Args:
    text: Sneakers kiss the court
Orange arc meets hush and hope
Nets breathe, crowd erupts
================================= Tool Message =================================
Name: check_haiku_lines

Correct, This haiku has 3 lines.
================================== Ai Message ==================================

Sneakers kiss the court
Orange arc meets hush and hope
Nets breathe, crowd erupts


### Other useful information

In [22]:
result

{'messages': [HumanMessage(content='Please write me a poem', additional_kwargs={}, response_metadata={}, id='ade4b22c-ca88-4510-bb12-ad670059f3a3'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 813, 'prompt_tokens': 170, 'total_tokens': 983, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 768, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CNngGKxxkDowI9Gvii7MMGD0gVOKr', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--2d3b84bb-26fa-4e2a-8076-4cf509acc928-0', tool_calls=[{'name': 'check_haiku_lines', 'args': {'text': 'Sneakers kiss the court\nOrange arc meets hush and hope\nNets breathe, crowd erupts'}, 'id': 'call_O4GrvFp6i55zxaUxCZWEXxGv', 'type': 'tool_call'}], u

In [23]:
result["messages"][-1]

AIMessage(content='Sneakers kiss the court\nOrange arc meets hush and hope\nNets breathe, crowd erupts', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 233, 'total_tokens': 256, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CNngQRZA9LMBep1IWW53f5ERkMyWF', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--81220a19-19cb-42e3-9bec-55ec6b29f71c-0', usage_metadata={'input_tokens': 233, 'output_tokens': 23, 'total_tokens': 256, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [24]:
result["messages"][-1].usage_metadata

{'input_tokens': 233,
 'output_tokens': 23,
 'total_tokens': 256,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 0}}

In [25]:
result["messages"][-1].response_metadata

{'token_usage': {'completion_tokens': 23,
  'prompt_tokens': 233,
  'total_tokens': 256,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
 'model_provider': 'openai',
 'model_name': 'gpt-5-2025-08-07',
 'system_fingerprint': None,
 'id': 'chatcmpl-CNngQRZA9LMBep1IWW53f5ERkMyWF',
 'service_tier': 'default',
 'finish_reason': 'stop',
 'logprobs': None}